In [ ]:
# default_exp core

# CircuitPython_pico_pi_controller

> The `pico_pi_controller` module contains CircuitPython classes to manage multiple Raspberry Pi SBCs from a Raspberry Pi Pico, using i2c.

## About

The classes in this module inherit from Adafruit's `busio` & `adafruit_bus_device` I2C classes for CircuitPython. `busio` is distributed with [CircuitPython](https://circuitpython.org), and `adafruit_bus_device` is distributed with the [CircuitPython Library Bundle](https://circuitpython.org/libraries).

Broadcom SoC hardware I2C is utilized both for the controlling MCU (the Pico) and the controlled SBCs (Raspberry Pi 4/3+/Zero). I2C baudrate is limited from the controller (the Pico) to reduce transmit/receive errors.

Controller programs may be written with hard-coded I2C peripheral addresses of connected SBCs (called 'devices' by these classes). Or, autodiscovery may be used so that devices may be added without changing software on the controller.

For example:

<code>
from pico_pi_controller import *
    
mycontroller = PPController()

mycontroller.addDevice('0x13')
mycontroller.listdevices()
</code>

This example will initialize I2C on the controller using the board defaults (I2C bus 1), retrieve host information & statistics from the SBC at I2C address 0x13, then return a list of device information.

## Module

In [ ]:
#hide
try:
    from nbdev.showdoc import *
except:
    pass
    """CircuitPython kernel has no nbdev"""

In [ ]:
#export
from sys import byteorder, modules
from time import sleep, struct_time
import board
#import busio
from adafruit_bus_device import i2c_device
try:
    from rtc import RTC
except:
    pass
try:
    from adafruit_datetime import datetime
except:
    from datetime import datetime
try:
    from adafruit_itertools.adafruit_itertools import chain
except:
    from itertools import chain 
    
try:
    import adafruit_logging as logging
    logger = logging.getLogger('PPC')
    logger.setLevel(logging.DEBUG)
except:
    import logging
    logger = logging.getLogger()
    logging.basicConfig(level = logging.DEBUG)
    
IDENTITY = bytearray(([ord(c) for c in list('ppcd')]))
"""identifier string used by RPi devices"""

CLR = bytearray(); CLR.append(ord('C')) # request clear transmit FIFO
IDF = bytearray(); IDF.append(ord('I')) # request to send [str] identification
HOS = bytearray(); HOS.append(ord('H')) # request to send [str] hostname
TIM = bytearray(); TIM.append(ord('T')) # request to send [int] datetime
BOS = bytearray(); BOS.append(ord('B')) # request to send [bool] bosmang status
LOD = bytearray(); LOD.append(ord('L')) # request to send [int] load
TZO = bytearray(); TZO.append(ord('Z')) # request to send [int] timezone (sec offset from UTC)
PEN = bytearray(); PEN.append(ord('P')) # request to send [int] MCU pin connected to RPi PEN

In [ ]:
#export
class PPDevice():
    """Represents an I2C peripheral device known to a `PPController`
    and stores data from those peripherals identified as PPC devices."""
    def __init__(self, controller=None, device_address=None, *argv, **kwargs):
        self.controller  = controller
        self.device_address = device_address
        self.device      = None
        """The I2CDevice creted by PPController."""
        
        self.retries     =  0
        self.retries_max = 10
        """retry count before I2CDevice is considered 'other', i.e. not a PPC device."""
        self.lastonline  = None
        """controller timestamp updated with each successful receive. reports & bosmang
           can decide what to do with this info."""
        
        """All data below are received *from* the PPC device:"""
        
        self.bosmang    = False
        """Declaration that device will send datetime & control instructions to controller.
           Only one bosmang per controller please, unless you wanya chaos."""
        self.UART_RX    = None
        """MCU GPIO RX for passthru from bosmang console TX"""
        self.UART_TX    = None
        """MCU GPIO TX for passthru from bosmang console RX"""
        self.PEN        = None
        """MCU GPIO connected to RPi PEN pin"""
        
        self.hostname   = None
        self.datetime   = None
        """datetime object converted from timestamp, used to send datetime as bosmang & 
           to check for datetime skew on other devices."""
        self.utcoffset  = None
        self.loadavg    = None
        
        self.IDstr      = type(self).__name__[0:3]+": "+str(hex(self.device_address))

    def logTXN(self, fname, message, msg=None):
        logger.info(type(self).__name__[0:3]+": "+
                    str(hex(self.device_address))+" "
                    "{:<27}".format(message+" "+str(msg or ''))+
                    "{:<13}".format(fname)+
                    self.controller.IDstr)
    
    def getHostname(self):
        fname='getHostname'
        self.logTXN(fname,"querying device")
        with self.device as i2cdevice:
            msg = bytearray(16)
            try:
                i2cdevice.write_then_readinto(CLR,msg)
                """Clear the i2c peripheral's transmit FIFO"""
            except:
                pass
            msg = bytearray(1)
            try:
                i2cdevice.write_then_readinto(HOS,msg)
                """Get the length in bytes of the hostname"""
                msg = bytearray(int.from_bytes(msg, byteorder))  
                i2cdevice.readinto(msg)
                self.logTXN(fname,"recvd hostname",msg.decode())
            except:
                pass
        return msg.decode()
 
    def getDatetime(self): 
        fname='getDatetime'
        self.logTXN(fname,"querying device")
        with self.device as i2cdevice:
            msg = bytearray(16)
            try:
                i2cdevice.write_then_readinto(CLR,msg)
                """Clear the i2c peripheral's transmit FIFO"""
            except:
                pass
            msg = bytearray(4)
            try:
                i2cdevice.write_then_readinto(TIM,msg)
                self.logTXN(fname,"recvd datetime",int.from_bytes(bytes(msg),byteorder))
                return datetime.fromtimestamp((int.from_bytes(bytes(msg),byteorder)))
            except: 
                pass
        return None
 
    def getBosmang(self): 
        fname='getBosmang'
        self.logTXN(fname,"querying device")
        with self.device as i2cdevice:
            msg = bytearray(16)
            try:
                i2cdevice.write_then_readinto(CLR,msg)
                """Clear the i2c peripheral's transmit FIFO"""
            except:
                pass
            msg = bytearray(1)
            try:
                i2cdevice.write_then_readinto(BOS,msg)
                #self.logTXN(fname,"recvd bosmang status",int.from_bytes(bytes(msg),byteorder))
                self.logTXN(fname,"recvd bosmang status:",bool(msg.decode()))
                return bool(int.from_bytes(bytes(msg),byteorder))
            except: 
                pass
        return None
 
    def getTimeZone(self): 
        fname='getTimeZone'
        self.logTXN(fname,"querying device")
        with self.device as i2cdevice:
            msg = bytearray(16)
            try:
                i2cdevice.write_then_readinto(CLR,msg)
                """Clear the i2c peripheral's transmit FIFO"""
            except:
                pass
            msg = bytearray(3)
            #try:
            i2cdevice.write_then_readinto(TZO,msg)
            self.logTXN(fname,"recvd utcoffset",int.from_bytes(bytes(msg),byteorder))
            return int.from_bytes(bytes(msg),byteorder)
            #except: 
            #    pass
        return None
 
    def getLoadavg(self): 
        return self.loadavg
 
    def getUART_RX_GPIO(self): 
        return self.UART_RX
 
    def getUART_TX_GPIO(self): 
        return self.UART_TX
 
    def getPEN_GPIO(self): 
        return self.PEN

## New heading

In [ ]:
#export
class PPController():
    """Represents one of the system's I2C busses and tracks which I2C
    peripherals are `PPDevice`s."""
    #def __init__(self, scl=board.SCL, sda=board.SDA, frequency=4800, timeout=10000, bosmang=None):
    def __init__(self, **kwargs):
        self.i2c = None
        #self.scl = scl
        #self.sda = sda
        #self.frequency=frequency
        #self.timeout=timeout
        print(kwargs)
        self.scl = kwargs.pop('scl', board.SCL)
        self.sda = kwargs.pop('sda', board.SDA)
        self.frequency = kwargs.pop('frequency', 4800)
        self.timeout = kwargs.pop('timeout', 10000)
        self.bosmang = kwargs.pop('bosmang', None)
        """PPDevice hostname selected to recieve datetime & control instructions from,
           have UART connected for passthru, etc. Default is `None`."""
        self.datetime = None
        """to receive datetime from bosmang & to check for datetime skew on other devices."""
        self.utcoffset = None
        self.clock = RTC()
        
        self.devices = []
        self.noident = []
        self.othrdev = []
        
        self.IDstr   = type(self).__name__[0:3]+":"+str(self.scl)+"/"+str(self.sda)
        self.IDstr   = str(self.scl).strip('board.')+"/"+str(self.sda).strip('board.')
        
    def logTXN(self, fname, message, hexaddr=None, msg=None):
        logger.info(type(self).__name__[0:3]+": "+
                    str(hexaddr or '    ')+" "
                    "{:<27}".format(message+" "+str(msg or ''))+
                    "{:<13}".format(fname)+
                    self.IDstr)
    
    def scanForNew(self):
        fname='scanForNew'
        while not self.i2c.try_lock():
            pass
        self.logTXN(fname,">>> SCANNING I2C bus <<<")
           
        for addr in self.i2c.scan():
            if not any(d.device_address == addr for d in chain(self.devices,self.noident,self.othrdev)):  
                self.noident.append(PPDevice(controller=self,device_address=addr))
                self.noident[-1].device=i2c_device.I2CDevice(self.i2c,device_address=addr,probe=False)
                self.logTXN(fname,"added I2C peripheral",hex(addr))
        self.i2c.unlock()
        return True

    def identifyNew(self):
        fname='identifyNew'
        for peripheral in self.noident:
            self.logTXN(fname,"querying I2C peripheral",hex(peripheral.device_address))
            with peripheral.device as unident:
                msg = bytearray(16)
                try:
                    unident.write_then_readinto(CLR,msg)
                    """Clear the i2c peripheral's transmit FIFO"""
                except:
                    pass
                msg = bytearray(len(IDENTITY))
                try:
                    unident.write_then_readinto(IDF,msg,in_end=len(IDENTITY))
                except:
                    self.logTXN(fname,"WRITE FAILED",hex(peripheral.device_address))
                    pass
                if msg == IDENTITY:
                    self.devices.append(self.noident.pop(self.noident.index(peripheral)))
                    self.logTXN(fname,">>>  added PPDevice <<<",hex(peripheral.device_address))
                else:
                    peripheral.retries += 1
                    self.logTXN(fname,"ID FAILED on retry",hex(peripheral.device_address),peripheral.retries)
                    if peripheral.retries >= peripheral.retries_max:
                        self.logTXN(fname,"max retries reached. Releasing.",hex(peripheral.device_address))
                        self.othrdev.append(self.noident.pop(self.noident.index(peripheral)))
                        del self.othrdev[-1].device
                
    def autoAddDevices(self):
        self.scanForNew()
        self.identifyNew()
                
    def listdevices(self):
        return self.devices
    
    def queryAll(self):
        fname='queryAll'
        for ppd in self.devices:
            ppd.hostname = ppd.getHostname() 
            ppd.datetime = ppd.getDatetime() 
            ppd.bosmang  = ppd.getBosmang() 
            if ppd.bosmang:
                self.clock.datetime = ppd.datetime.timetuple()
                self.logTXN('',str(datetime.now()))
            ppd.utcoffset = ppd.getTimeZone() 
            ppd.loadavg = ppd.getLoadavg()
            ppd.UART_RX = ppd.getUART_RX_GPIO()
            ppd.UART_TX = ppd.getUART_TX_GPIO()
            ppd.PEN = ppd.getPEN_GPIO()


In [ ]:
#hide
#try:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))
sleep(0.2)
from nbdev.export import notebook2script
notebook2script()
#except:
#    pass
#    """CircuitPython kernel has no nbdev"""

!!echo -e "\x02\x04" | tee /dev/ttyACM0

%cp -v CircuitPython_pico_pi_controller/core.py /CIRCUITPY/lib/CircuitPython_pico_pi_controller.py

<IPython.core.display.Javascript object>

Converted 00_core.ipynb.
Converted 10_reports.ipynb.
Converted index.ipynb.
'CircuitPython_pico_pi_controller/core.py' -> '/CIRCUITPY/lib/CircuitPython_pico_pi_controller.py'
